<a href="https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Email_Sorter_Python_Application_(Version_38_09_Priority_Sorting_Implemented).ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Python_Application_(Version_38_09_Priority_Sorting_Implemented).ipynb
"""

import os
import win32com.client
import pandas as pd
import datetime
import openpyxl
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
from tkcalendar import Calendar
import threading
import logging
import time
import json
import re
import pythoncom

class EmailSorter:
    """
    A class to sort emails in Outlook based on rules defined in an Excel file
    and configuration from a JSON file.
    Supports live monitoring and bulk processing modes.
    Version 38.09: Implements strict rule priority based on the numeric prefix
                   of the sheet name (e.g., '1.ToDelete-KW1' has priority 1).
                   Fully supports multi-column keyword loading.
    """

    # Define config file path once here for consistency
    # Assuming the config file is named 'config.json' as per the previous step
    CONFIG_FILE_NAME = 'config.json'

    def __init__(self, config_path=None):
        """
        Initializes the EmailSorter with configuration, sets up paths,
        and initializes internal state variables.
        """
        self.config_path = config_path if config_path else self.CONFIG_FILE_NAME
        self.config = self.load_config()
        self.outlook = None
        self.namespace = None
        self.inbox = None
        self.outlook_thread = None
        self.live_mode_running = threading.Event()
        self.last_run_time = None
        self.rules = {} # Stores all rule data
        self.ordered_rule_names = [] # Stores rule names sorted by priority
        self.smtp_cache = {}
        self.folder_cache = {}
        self.logs_dir = os.path.dirname(self.config_path) if self.config_path else os.getcwd()
        self.live_logger = self.setup_logger("LiveLog", self.config.get('log_live_path', 'movemail_live.log'))
        self.bulk_logger = self.setup_logger("BulkLog", self.config.get('log_bulk_path', 'movemail_bulk.log'))
        self.invalid_logger = self.setup_logger("InvalidLog", self.config.get('log_invalid_path', 'movemail_invalid.log'))
        self.app_start_time = datetime.datetime.now()

    def setup_logger(self, name, log_path):
        """Configures and returns a file logger."""
        logger = logging.getLogger(name)
        logger.setLevel(logging.INFO)

        # Ensure log directory exists
        log_dir = os.path.dirname(log_path)
        if log_dir and not os.path.exists(log_dir):
            try:
                os.makedirs(log_dir)
            except OSError as e:
                # Fallback if directory creation fails (e.g., permission error)
                print(f"Warning: Could not create log directory {log_dir}. Logging to current working directory. Error: {e}")
                log_path = os.path.join(os.getcwd(), os.path.basename(log_path))

        # Create file handler
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s||%(levelname)s||%(message)s'))

        # Prevent adding multiple handlers if called multiple times
        if not logger.handlers:
            logger.addHandler(file_handler)

        return logger

    def load_config(self):
        """Loads configuration from the JSON file."""
        try:
            with open(self.config_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"Error: Configuration file not found at {self.config_path}")
            # Fallback to a minimal structure to prevent immediate crash
            return {'sheet_map': {}, 'xls_path': '', 'log_live_path': '', 'log_bulk_path': '', 'log_invalid_path': ''}
        except json.JSONDecodeError as e:
            print(f"Error: Invalid JSON format in {self.config_path}. Error: {e}")
            return {'sheet_map': {}, 'xls_path': '', 'log_live_path': '', 'log_bulk_path': '', 'log_invalid_path': ''}

    def init_outlook(self):
        """Initializes the Outlook COM object."""
        try:
            pythoncom.CoInitialize() # Initialize COM for the current thread
            self.outlook = win32com.client.Dispatch("Outlook.Application")
            self.namespace = self.outlook.GetNamespace("MAPI")
            self.inbox = self.namespace.GetDefaultFolder(6) # olFolderInbox = 6
            return True
        except Exception as e:
            self.invalid_logger.critical(f"OutlookInitError||init_outlook|Failed to initialize Outlook COM object: {e}")
            return False

    def load_rules(self):
        """
        Loads rules from the Excel file, supports multi-column loading, and
        DETERMINES PRIORITY based on the sheet name's numeric prefix.
        """
        if not self.config.get('xls_path') or not os.path.exists(self.config['xls_path']):
            self.invalid_logger.error(f"RuleLoadingError||load_rules|Excel path missing or file not found: {self.config.get('xls_path')}")
            return

        self.rules = {}
        self.smtp_cache = {}

        # Load data from the sheets
        for rule_name, rule_config in self.config['sheet_map'].items():
            sheet_name = rule_config['sheet']

            column_key = rule_config.get('column')
            columns_list = rule_config.get('columns')

            column_names_to_read = None

            if column_key is not None:
                column_names_to_read = [column_key]
            elif columns_list and isinstance(columns_list, list) and columns_list:
                column_names_to_read = columns_list

            if rule_name == 'SMTPResolutionCache':
                try:
                    df_cache = pd.read_excel(
                        self.config['xls_path'],
                        sheet_name=sheet_name,
                        header=0,
                        usecols=['Sender', 'ResolvedFolder']
                    )

                    df_cache = df_cache.dropna(subset=['Sender'])
                    self.smtp_cache = dict(zip(
                        df_cache['Sender'].astype(str).str.upper(),
                        df_cache['ResolvedFolder'].astype(str)
                    ))
                    self.live_logger.info(f"Loaded SMTP Cache with {len(self.smtp_cache)} entries.")

                except Exception as e:
                    self.invalid_logger.critical(f"CacheLoadingError||load_rules|Failed to load SMTPResolutionCache sheet. Error: {e}")
                continue # Skip rest of loop for cache sheet

            # Non-cache rules processing
            if column_names_to_read:
                match_field = rule_config.get('match_field', 'sender_email')
                destination_name = rule_config.get('destination_name', None)

                # --- NEW: Derive Priority from Sheet Name ---
                priority_match = re.match(r'(\d+)', sheet_name)
                # Default priority (999) if no number is found, ensuring it runs last
                priority = int(priority_match.group(1)) if priority_match else 999

                if destination_name and column_names_to_read:
                    try:
                        df = pd.read_excel(
                            self.config['xls_path'],
                            sheet_name=sheet_name,
                            header=0,
                            usecols=column_names_to_read
                        )

                        df_clean = df.apply(lambda x: x.astype(str).str.strip().str.upper())

                        rule_list = []
                        for col in column_names_to_read:
                            if col in df_clean.columns:
                                rule_list.extend(df_clean[col].dropna().tolist())
                            else:
                                self.invalid_logger.warning(f"RuleLoadingWarning||load_rules|Column '{col}' not found in sheet '{sheet_name}'. Skipping column.")

                        rule_list = list(set(rule_list))

                        if rule_list:
                            self.rules[rule_name] = {
                                'priority': priority, # Store the determined priority
                                'match_field': match_field,
                                'destination': destination_name,
                                'keywords': rule_list
                            }
                            self.live_logger.info(f"Rule Loaded: {rule_name} (Priority {priority}) from sheet {sheet_name} with {len(rule_list)} unique keywords.")

                    except Exception as e:
                        self.invalid_logger.critical(f"RuleLoadingError||load_rules|Failed to load sheet {sheet_name}. Error: {e}")

        # --- FINAL STEP: Create the ordered list of rule names ---
        # Sort rule names based on the 'priority' field in the rule data
        self.ordered_rule_names = sorted(
            [name for name, data in self.rules.items() if 'priority' in data],
            key=lambda name: self.rules[name]['priority']
        )
        self.live_logger.info(f"Rule Execution Order: {self.ordered_rule_names}")


    def find_outlook_folder(self, folder_path):
        """
        Dynamically finds or creates the target Outlook folder based on the path (e.g., 'DACS-My').
        Uses a cache to avoid repeated lookups.
        """
        if folder_path in self.folder_cache:
            return self.folder_cache[folder_path]

        try:
            # Split path by backslash for nested folders (e.g., 'Trade&BO\Trade')
            path_parts = folder_path.split('\\')
            current_folder = self.inbox

            for part in path_parts:
                found = False
                for subfolder in current_folder.Folders:
                    if subfolder.Name == part:
                        current_folder = subfolder
                        found = True
                        break

                if not found:
                    # Folder does not exist, create it
                    current_folder = current_folder.Folders.Add(part)
                    self.live_logger.info(f"FolderCreation||find_outlook_folder|Created folder: {current_folder.FolderPath}")

            # Cache and return the final folder object
            self.folder_cache[folder_path] = current_folder
            return current_folder

        except Exception as e:
            self.invalid_logger.critical(f"FolderLookupError||find_outlook_folder|Failed to find or create folder '{folder_path}'. Error: {e}")
            return None

    def get_email_body_text(self, mail_item):
        """
        Extracts and cleans text from the email body, prioritizing HTML and falling
        back to plain text.
        """
        try:
            # 1. Try to get text from the HTMLBody (preferred)
            if mail_item.HTMLBody:
                # Use regex to strip HTML tags and normalize whitespace
                html_body = mail_item.HTMLBody
                text = re.sub('<[^<]+?>', ' ', html_body)
                # Normalize whitespace (replace multiple spaces/newlines with single space)
                text = re.sub(r'\s+', ' ', text).strip()
                return text.upper()

            # 2. Fallback to Plain Text Body
            if mail_item.Body:
                return mail_item.Body.strip().upper()

            return ""
        except Exception as e:
            self.invalid_logger.error(f"BodyParseError||get_email_body_text|Error parsing body for email '{mail_item.Subject}': {e}")
            return ""

    def process_email(self, mail_item, log_func):
        """Applies rules to a single email item based on priority order."""

        # Ensure COM is initialized for the thread
        try:
            pythoncom.CoInitialize()
        except:
            pass # Already initialized

        # Prepare email data
        sender_address = mail_item.SenderEmailAddress.upper()
        subject = mail_item.Subject.upper()
        body_text = None # Lazy load body

        resolved_folder = None

        # 1. SMTP Cache Lookup (Highest Priority)
        if sender_address in self.smtp_cache:
            resolved_folder_path = self.smtp_cache[sender_address]
            resolved_folder = self.find_outlook_folder(resolved_folder_path)

            if resolved_folder:
                log_func(f"MatchCache||{resolved_folder_path}||{sender_address}||{subject}")
            else:
                self.invalid_logger.error(f"CacheDestinationError||{resolved_folder_path}||{sender_address}||{subject}||Folder not found for cached entry. Skipping move.")
                return # Skip move if cached folder is invalid

        # 2. Rules Execution (If not found in cache, follow priority order)
        if not resolved_folder:

            # Iterate using the pre-sorted list of rule names
            for rule_name in self.ordered_rule_names:
                rule_data = self.rules[rule_name]

                match_field = rule_data['match_field']
                destination_name = rule_data['destination']
                keywords = rule_data['keywords']

                # Determine the text to match against
                text_to_match = ""
                if match_field == 'sender_email':
                    text_to_match = sender_address
                elif match_field == 'subject_only':
                    text_to_match = subject
                elif match_field == 'subject_and_body':
                    if body_text is None:
                        body_text = self.get_email_body_text(mail_item)
                    # Match against Subject + Body
                    text_to_match = subject + " " + body_text

                # Perform the matching (substring match)
                for keyword in keywords:
                    if keyword in text_to_match:
                        resolved_folder_path = destination_name
                        resolved_folder = self.find_outlook_folder(resolved_folder_path)

                        if resolved_folder:
                            log_func(f"MatchRule||{resolved_folder_path}||{sender_address}||{subject}||Rule: {rule_name} (Priority {rule_data['priority']}), Keyword: {keyword}")

                            # Update SMTP Cache with the newly found rule destination
                            self.smtp_cache[sender_address] = resolved_folder_path
                        else:
                            self.invalid_logger.error(f"RuleDestinationError||{resolved_folder_path}||{sender_address}||{subject}||Rule: {rule_name}, Destination folder not found. Skipping move.")

                        break # Rule matched, stop checking keywords for this rule

                if resolved_folder:
                    break # Email classified, stop checking rules

        # 3. Move the email item
        if resolved_folder:
            try:
                # Use the Move method (robust against different item types)
                mail_item.Move(resolved_folder)
                return True
            except Exception as e:
                # Log critical failure to move the item
                self.invalid_logger.critical(f"MoveFailure||{resolved_folder_path}||{sender_address}||{subject}||Error moving item: {e}")
                return False
        else:
            # Log as unclassified
            log_func(f"NoMatch||Unclassified||{sender_address}||{subject}")
            return False

    # --- Live Mode Logic ---

    def run_live_mode_loop(self):
        """The main loop for live email monitoring in a separate thread."""

        # Load rules and initialize outlook in the thread
        if not self.init_outlook():
            self.live_logger.critical("LiveModeExit||Outlook failed to initialize. Exiting live mode thread.")
            return

        self.load_rules()

        self.last_run_time = self.get_initial_start_time()
        self.live_logger.info(f"LiveModeStart||Initial check time set to: {self.last_run_time.strftime('%Y-%m-%d %H:%M:%S')}")

        while self.live_mode_running.is_set():

            # Run the processing logic
            current_time = datetime.datetime.now()

            # Check for midnight reset (for rules applied to older emails daily)
            if self.should_reset_daily(current_time):
                self.live_logger.info("DailyReset||Midnight reset triggered. Reprocessing all emails since app start.")
                # Reset last_run_time to app_start_time for a full daily reprocessing
                process_start_time = self.app_start_time
                self.last_run_time = current_time
            else:
                # Normal hourly check
                process_start_time = self.last_run_time
                self.last_run_time = current_time # Update immediately after determining start time

            self.process_inbox(process_start_time)

            # Wait for 60 minutes or until stop event is set
            self.live_mode_running.wait(3600) # Wait for 1 hour

        self.live_logger.info("LiveModeStop||Live mode thread finished.")
        # Ensure COM is uninitialized if necessary, though Python usually handles it.

    def get_initial_start_time(self):
        """Determines the historical start time for the first run."""
        # 1. Use the last saved run time from the log file (if possible)
        try:
            with open(self.config['log_live_path'], 'r') as f:
                last_line = None
                for line in f:
                    if "LiveModeCheck" in line or "LiveModeStart" in line:
                        last_line = line

                if last_line:
                    # Example log format: 2023-10-26 14:00:00,000||INFO||LiveModeCheck
                    dt_str = last_line.split('||')[0].split(',')[0].strip()
                    last_dt = datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
                    # Start 1 second after the last check to avoid reprocessing the last batch
                    return last_dt + datetime.timedelta(seconds=1)
        except Exception as e:
            self.live_logger.warning(f"LogReadError||get_initial_start_time|Could not read last run time from log: {e}")

        # 2. Fallback to 24 hours ago
        return datetime.datetime.now() - datetime.timedelta(hours=24)

    def should_reset_daily(self, current_time):
        """Checks if the run should be a daily reset (first run after midnight)."""
        if self.last_run_time is None:
            # Should be handled by get_initial_start_time, but safety check.
            return False

        # Check if the date has changed since the last run
        return current_time.day != self.last_run_time.day

    def process_inbox(self, start_date):
        """Processes emails in the Inbox since start_date."""

        # Ensure COM is initialized for the thread
        try:
            pythoncom.CoInitialize()
        except:
            pass # Already initialized

        self.live_logger.info(f"LiveModeCheck||Starting check for emails received after: {start_date.strftime('%Y-%m-%d %H:%M:%S')}")

        # Convert Python datetime to Outlook filter string format
        # Outlook filter requires date in a specific format enclosed in quotes
        date_filter = f"[ReceivedTime] > '{start_date.strftime('%m/%d/%Y %H:%M %p')}'"

        try:
            # Filter items in the inbox
            items = self.inbox.Items.Restrict(date_filter)
            items.Sort("[ReceivedTime]", False) # Sort descending for newest first

            processed_count = 0

            # Process items (iterate backwards to avoid issues if items are moved)
            for i in range(items.Count, 0, -1):
                mail_item = items[i]
                # Check if the item is a MailItem (Type 43) before processing
                if mail_item.Class == 43: # olMail
                    if self.process_email(mail_item, self.live_logger.info):
                        processed_count += 1

            self.live_logger.info(f"LiveModeCheck||Completed. Processed {items.Count} items, moved {processed_count} emails.")

        except Exception as e:
            self.invalid_logger.critical(f"LiveProcessError||process_inbox|Failed to process Inbox items: {e}")

    def start_live_mode(self, root):
        """Starts the live monitoring thread."""
        if self.live_mode_running.is_set():
            messagebox.showinfo("Live Mode", "Live Mode is already running.")
            return

        # Check if rules are loaded and Outlook is initialized
        if not self.rules:
            # Attempt a fresh load before starting
            self.load_rules()
            if not self.rules:
                 messagebox.showerror("Error", "Failed to load rules from Excel. Cannot start Live Mode.")
                 return

        # Start the thread
        self.live_mode_running.set()
        self.outlook_thread = threading.Thread(target=self.run_live_mode_loop)
        self.outlook_thread.daemon = True # Allows program to exit if main thread stops
        self.outlook_thread.start()

        # Disable the Start button and enable the Stop button (assuming GUI elements exist)
        # Note: In a production setup, you would use Tkinter controls to update the GUI state.

        # Display status message in GUI
        status_label = root.nametowidget("status_label")
        status_label.config(text="Status: Live Mode Running...", fg="green")

        messagebox.showinfo("Live Mode", "Live Mode started successfully. It will now check for emails hourly.")

    def stop_live_mode(self, root):
        """Stops the live monitoring thread."""
        if self.live_mode_running.is_set():
            self.live_mode_running.clear()
            self.outlook_thread.join() # Wait for the thread to finish cleanly

            # Display status message in GUI
            status_label = root.nametowidget("status_label")
            status_label.config(text="Status: Live Mode Stopped.", fg="red")

            messagebox.showinfo("Live Mode", "Live Mode stopped.")
        else:
            messagebox.showinfo("Live Mode", "Live Mode is not running.")

    # --- Bulk Mode Logic ---

    def run_bulk_mode(self, root, start_date_str, end_date_str):
        """Processes emails in a selected date range."""

        # Disable GUI elements during processing
        root.config(cursor="wait")
        status_label = root.nametowidget("status_label")
        status_label.config(text="Status: Running Bulk Mode...", fg="orange")
        root.update()

        try:
            # 1. Input Validation and Date Parsing
            start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
            end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

            # Bulk mode processes until the end of the end_date day
            end_date = end_date + datetime.timedelta(days=1) - datetime.timedelta(seconds=1)

            if start_date >= end_date:
                messagebox.showerror("Error", "Start date must be before the end date.")
                return

            # 2. Initialization (in the same thread for simplicity in bulk mode)
            if not self.init_outlook():
                messagebox.showerror("Error", "Failed to initialize Outlook. Cannot run Bulk Mode.")
                return

            self.load_rules()

            if not self.rules:
                 messagebox.showerror("Error", "Failed to load rules from Excel. Cannot run Bulk Mode.")
                 return

            self.bulk_logger.info(f"BulkModeStart||Processing period: {start_date_str} to {end_date_str}")

            # 3. Build Filter String
            start_filter_str = start_date.strftime('%m/%d/%Y %I:%M %p')
            end_filter_str = end_date.strftime('%m/%d/%Y %I:%M %p')

            date_filter = f"[ReceivedTime] >= '{start_filter_str}' AND [ReceivedTime] <= '{end_filter_str}'"

            # 4. Get and Process Items
            items = self.inbox.Items.Restrict(date_filter)
            items.Sort("[ReceivedTime]", False) # Sort descending (optional, but good practice)

            total_items = items.Count
            processed_count = 0

            # Process items (iterate backwards to avoid issues if items are moved)
            for i in range(total_items, 0, -1):
                mail_item = items[i]
                if mail_item.Class == 43: # olMail
                    if self.process_email(mail_item, self.bulk_logger.info):
                        processed_count += 1

                # Update GUI progress (simple message)
                status_label.config(text=f"Status: Bulk Mode Running... {total_items - i + 1}/{total_items} processed.")
                root.update()

            self.bulk_logger.info(f"BulkModeEnd||Completed. Processed {total_items} items, moved {processed_count} emails.")
            messagebox.showinfo("Bulk Mode", f"Bulk Mode completed.\nProcessed {total_items} emails.\nMoved {processed_count} emails.")

        except ValueError:
            messagebox.showerror("Error", "Invalid date format. Please use YYYY-MM-DD.")
        except Exception as e:
            self.invalid_logger.critical(f"BulkProcessError||run_bulk_mode|General error during bulk processing: {e}")
            messagebox.showerror("Error", f"An unexpected error occurred during Bulk Mode: {e}")
        finally:
            # Re-enable GUI elements
            root.config(cursor="")
            status_label.config(text="Status: Ready.", fg="blue")
            root.update()

    def save_smtp_cache(self):
        """Saves the current SMTP cache back to the Excel file."""
        if not self.smtp_cache:
            self.live_logger.info("CacheSaveSkip||No SMTP cache data to save.")
            return

        try:
            # 1. Prepare data for DataFrame
            cache_list = [
                {'Sender': sender.title(), 'ResolvedFolder': folder}
                for sender, folder in self.smtp_cache.items()
            ]
            df_cache = pd.DataFrame(cache_list)

            # 2. Get the sheet name and file path
            cache_config = self.config['sheet_map'].get('SMTPResolutionCache', {})
            sheet_name = cache_config.get('sheet', 'SMTPResolutionCache')
            xls_path = self.config['xls_path']

            if not os.path.exists(xls_path):
                 self.invalid_logger.error(f"CacheSaveError||save_smtp_cache|Excel file not found at {xls_path}. Cannot save cache.")
                 return

            # 3. Load the workbook and replace the sheet
            with pd.ExcelWriter(xls_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                # Note: Using mode='a' and if_sheet_exists='replace' correctly overwrites only the specified sheet.
                df_cache.to_excel(writer, sheet_name=sheet_name, index=False)

            self.live_logger.info(f"CacheSaveSuccess||Saved {len(self.smtp_cache)} entries to {sheet_name} in {xls_path}.")

        except Exception as e:
            self.invalid_logger.critical(f"CacheSaveError||save_smtp_cache|Failed to save SMTP cache to Excel: {e}")
            messagebox.showerror("Cache Error", f"Failed to save SMTP cache: {e}. Check log for details.")


    # --- GUI Logic ---

    def start_gui(self):
        """Sets up and runs the main Tkinter GUI."""
        root = tk.Tk()
        root.title("Email Sorter v38.09 (Live/Bulk)")
        root.geometry("400x450")
        root.resizable(False, False)
        root.configure(bg="#f0f0f0")

        # Initialization check and prompt
        if not self.init_outlook():
            messagebox.showerror("Fatal Error", "Could not connect to Outlook. Please ensure Outlook is running and try again.")
            return

        # Load rules only once at startup
        self.load_rules()
        if not self.rules:
            messagebox.showwarning("Warning", "No rules were loaded from the Excel file. Please check 'sheet_map' in config and Excel data.")

        # Header Label
        header_label = tk.Label(root, text="Outlook Mailbox Sorter",
                                font=("Arial", 16, "bold"), bg="#f0f0f0", fg="#004d40")
        header_label.pack(pady=20)

        # Status Label (named for easy access in other methods)
        status_label = tk.Label(root, text="Status: Ready.", name="status_label",
                                font=("Arial", 10, "italic"), fg="blue", bg="#f0f0f0")
        status_label.pack(pady=5)

        # --- Live Mode Buttons ---

        # Frame for Live Mode buttons
        live_frame = tk.Frame(root, bg="#f0f0f0")
        live_frame.pack(pady=10)

        def start_live():
            self.start_live_mode(root)

        def stop_live():
            self.stop_live_mode(root)

        # Start Live Button
        start_live_btn = tk.Button(live_frame, text="Start Live Mode", command=start_live,
                                   bg="#4CAF50", fg="white", width=15, height=2,
                                   font=("Arial", 11, "bold"), relief=tk.RAISED)
        start_live_btn.pack(side=tk.LEFT, padx=10)

        # Stop Live Button
        stop_live_btn = tk.Button(live_frame, text="Stop Live Mode", command=stop_live,
                                  bg="#f44336", fg="white", width=15, height=2,
                                  font=("Arial", 11, "bold"), relief=tk.RAISED)
        stop_live_btn.pack(side=tk.RIGHT, padx=10)

        # --- Bulk Mode Logic and UI ---

        def pick_bulk():
            """Opens the date picker dialog for Bulk Mode."""

            # Dialog window setup
            bulk_window = tk.Toplevel(root)
            bulk_window.title("Bulk Mode Date Range")
            bulk_window.geometry("350x450")
            bulk_window.resizable(False, False)
            bulk_window.transient(root) # Keep on top of main window

            tk.Label(bulk_window, text="Select Start Date", font=("Arial", 12, "bold")).pack(pady=10)
            cal_start = Calendar(bulk_window, selectmode='day', date_pattern='yyyy-mm-dd',
                                 font="Arial 10", background="#004d40", foreground="white",
                                 selectbackground="#4CAF50", normalbackground="white")
            cal_start.pack(pady=5)

            tk.Label(bulk_window, text="Select End Date", font=("Arial", 12, "bold")).pack(pady=10)

            # Option to use today's date as end date
            end_today_var = tk.BooleanVar(value=True)

            def toggle_end_date_entry():
                if end_today_var.get():
                    end_date_entry.config(state=tk.DISABLED, fg='gray')
                    end_date_entry.delete(0, tk.END)
                else:
                    end_date_entry.config(state=tk.NORMAL, fg='black')

            # End Date Entry (defaults to today's date if checkbox is unchecked)
            today_str = datetime.datetime.now().strftime('%Y-%m-%d')
            end_date_entry = tk.Entry(bulk_window, width=15, font=("Arial", 10), justify='center', state=tk.DISABLED, fg='gray')
            end_date_entry.insert(0, today_str)
            end_date_entry.pack(pady=5)

            def set_end_date_from_cal():
                if not end_today_var.get():
                    end_date_entry.delete(0, tk.END)
                    end_date_entry.insert(0, cal_end.get_date())

            cal_end = Calendar(bulk_window, selectmode='day', date_pattern='yyyy-mm-dd',
                                 font="Arial 10", background="#004d40", foreground="white",
                                 selectbackground="#4CAF50", normalbackground="white", command=set_end_date_from_cal)

            end_today_check = tk.Checkbutton(bulk_window, text="End Date as Today",
                                             variable=end_today_var, command=toggle_end_date_entry)
            end_today_check.pack(pady=5)

            def start_bulk():
                start_date_val = cal_start.get_date()
                end_date_val = datetime.datetime.now().strftime('%Y-%m-%d') if end_today_var.get() else end_date_entry.get()

                # Close dialog and run in main window
                bulk_window.destroy()

                # Run bulk mode in a thread to prevent freezing the main GUI
                bulk_thread = threading.Thread(target=self.run_bulk_mode, args=(root, start_date_val, end_date_val))
                bulk_thread.daemon = True
                bulk_thread.start()

            tk.Button(bulk_window, text="Run Bulk Mode", command=start_bulk,
                      bg="#ff9800", fg="white", width=15, height=2,
                      font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=15)

        # Bulk Button
        tk.Button(root, text="Run Bulk Mode", command=pick_bulk,
                  bg="#ffc107", fg="white", width=18, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=8)

        footer_label = tk.Label(root, text="Live: Monitors emails based on smart schedule | Bulk: Process selected date range",
                                 font=("Arial", 8), fg="gray")
        footer_label.pack(side=tk.BOTTOM, pady=10)

        def on_closing():
            # This prompt also uses the robust save_smtp_cache
            if messagebox.askyesno("Exit", "Do you want to save the SMTP cache before exiting? (Recommended)"):
                self.save_smtp_cache()
            root.destroy()

        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()

def main():
    """Main function to run the Email Sorter application."""
    sorter = None
    try:
        # Use the class-level constant for config_path
        sorter = EmailSorter()
        sorter.start_gui()
    except Exception as e:
        print(f"Error starting Email Sorter application: {e}")
        if sorter and sorter.invalid_logger:
            sorter.invalid_logger.critical(f"AppStartupError||main|{e}")
    finally:
        # Final attempt to save cache if main GUI closes in an unexpected way,
        # but the prompt will handle it for user-initiated closes.
        if sorter:
            pass # No longer call save_smtp_cache directly here,

if __name__ == '__main__':
    main()